## A "Greedy" change approach

In [3]:
def greedyChange(amount, denominations):
    # Goal is to produce the fewest coins to achieve
    # given target "amount"
    # Strategy: Give as many of the largest coin 
    # denomination that is less than amount. 
    solution = []
    for coin in denominations:
        i = amount // coin       # truncating integer divide
        solution.append(i)
        amount -= coin * i
    return solution

s1 = greedyChange(72, [25,10,5,1])
print(s1, sum(s1))
s2 = greedyChange(40, [25,10,5,1])
print(s2, sum(s2))
s3 = greedyChange(40, [25,20,10,5,1])
print(s3, sum(s3))

[2, 2, 0, 2] 6
[1, 1, 1, 0] 3
[1, 0, 1, 1, 0] 3


## Another Approach

In [14]:
def exhaustiveChange(amount, denominations):
    bestN = 100
    count = [0 for i in range(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < 100):
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        value = sum([count[i]*denominations[i] for i in range(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in range(len(denominations))]
                bestN = n
    return solution

%time print(exhaustiveChange(40,[25,20,10,5,1]))

[0, 2, 0, 0, 0]
CPU times: user 699 ms, sys: 0 ns, total: 699 ms
Wall time: 701 ms


## Correct, but costly
* Our algorithm now gets the right answer for every value 1..100
* It must, because it considers every possible answer<br>(that’s the good thing about brute force)
* There is a downside though

In [16]:
%time print(exhaustiveChange(40, [25,10,5,1]))
%time print(exhaustiveChange(40, [25,20,10,5,1]))
%time print(exhaustiveChange(40, [13,11,7,5,3,1]))

[1, 1, 1, 0]
CPU times: user 155 ms, sys: 0 ns, total: 155 ms
Wall time: 149 ms
[0, 2, 0, 0, 0]
CPU times: user 632 ms, sys: 0 ns, total: 632 ms
Wall time: 628 ms
[0, 3, 1, 0, 0, 0]
CPU times: user 2min 50s, sys: 0 ns, total: 2min 50s
Wall time: 2min 50s


## Other tricks?

A Branch-and-bound algorithm

In [18]:
def branchAndBoundChange(amount, denominations):
    bestN = amount
    count = [0 for i in range(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < amount):             # Set upper bound to amount rather than 100
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        if (n > bestN):                                   # don't compute the amount if there are too many coins
            continue
        value = sum([count[i]*denominations[i] for i in range(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in range(len(denominations))]
                bestN = n
    return solution

%time print(branchAndBoundChange(40, [13,11,7,5,3,1]))

[0, 3, 1, 0, 0, 0]
CPU times: user 329 ms, sys: 0 ns, total: 329 ms
Wall time: 310 ms


## A Recursive Coin-Change Algorithm

In [ ]:
def RecursiveChange(M, c):
    if (M == 0):
        return [0 for i in range(len(c))]
    smallestNumberOfCoins = M+1
    for i in range(len(c)):
        if (M >= c[i]):
            thisChange = RecursiveChange(M - c[i], c)
            thisChange[i] += 1
            if (sum(thisChange) < smallestNumberOfCoins):
                bestChange = thisChange
                smallestNumberOfCoins = sum(thisChange)
    return bestChange

%time print(RecursiveChange(99, [13,11,7,5,3,1]))

## Change via Dynamic Programming

In [15]:
def DPChange(M, c):
    change = [[0 for i in range(len(c))]]
    for m in range(1,M+1):
        bestNumCoins = m+1
        for i in range(len(c)):
            if (m >= c[i]):
                thisChange = [x for x in change[m - c[i]]]
                thisChange[i] += 1
                if (sum(thisChange) < bestNumCoins):
                    change[m:m] = [thisChange]
                    bestNumCoins = sum(thisChange)
    return change[M]

%time print(DPChange(40, [1,3,5,7,11,13]))
%time print(DPChange(40, [1,3,5,7,11,13,17]))
%time print(DPChange(40, [1,3,5,7,11,13,17,19]))

[1, 0, 0, 0, 0, 3]
CPU times: user 692 µs, sys: 0 ns, total: 692 µs
Wall time: 540 µs
[1, 0, 1, 0, 0, 0, 2]
CPU times: user 332 µs, sys: 0 ns, total: 332 µs
Wall time: 329 µs
[2, 0, 0, 0, 0, 0, 0, 2]
CPU times: user 350 µs, sys: 0 ns, total: 350 µs
Wall time: 335 µs


## A Hybrid Approach: Memoization

In [3]:
change = {}                                            # This is a cache for saving bestChange[M]

def MemoizedChange(M, c):
    global change
    if (M in change):                                   # Check the cache first
        return [v for v in change[M]]
    if (len(change) == 0):                              # Initialize cache
        change[0] = [0 for i in range(len(c))]
    smallestNumberOfCoins = M+1
    for i in range(len(c)):
        if (M >= c[i]):
            thisChange = MemoizedChange(M - c[i], c)
            thisChange[i] += 1
            if (sum(thisChange) < smallestNumberOfCoins):
                bestChange = [v for v in thisChange]
                smallestNumberOfCoins = sum(thisChange)
    change[M] = [v for v in bestChange]                 # Add new M to cache 
    return bestChange

%time print(MemoizedChange(99, [1,7,42]))

[1, 2, 2]
CPU times: user 703 µs, sys: 0 ns, total: 703 µs
Wall time: 664 µs
